In [196]:
#导入常用库
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn as sk
#设置显示中文
plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False
# 无视警告
import warnings 
# warnings.filterwarnings('always') # 显示所有警告
warnings.filterwarnings('ignore') # 忽略所有警告

In [197]:
# 读取数据
data= pd.read_csv('US-pumpkins.csv')
data

,City Name,Type,Package,Variety,Sub Variety,Grade,Date,Low Price,High Price,Mostly Low,...,Unit of Sale,Quality,Condition,Appearance,Storage,Crop,Repack,Trans Mode,Unnamed: 24,Unnamed: 25
0,BALTIMORE,NaN,24 inch bins,NaN,NaN,NaN,4/29/17,270.00,280.00,270.0,...,NaN,NaN,NaN,NaN,NaN,NaN,E,NaN,NaN,NaN
1,BALTIMORE,NaN,24 inch bins,NaN,NaN,NaN,5/6/17,270.00,280.00,270.0,...,NaN,NaN,NaN,NaN,NaN,NaN,E,NaN,NaN,NaN
2,BALTIMORE,NaN,24 inch bins,HOWDEN TYPE,NaN,NaN,9/24/16,160.00,160.00,160.0,...,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN
3,BALTIMORE,NaN,24 inch bins,HOWDEN TYPE,NaN,NaN,9/24/16,160.00,160.00,160.0,...,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN
4,BALTIMORE,NaN,24 inch bins,HOWDEN TYPE,NaN,NaN,11/5/16,90.00,100.00,90.0,...,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1752,ST. LOUIS,NaN,22 lb cartons,MINIATURE,FLAT TYPE,NaN,9/30/16,14.75,14.75,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,LOWER.
1753,ST. LOUIS,NaN,36 inch bins,MINIATURE,ROUND TYPE,NaN,9/26/16,275.00,275.00,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,LOWER.
1754,ST. LOUIS,NaN,36 inch bins,MINIATURE,ROUND TYPE,NaN,9/27/16,275.00,275.00,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,STEADY.
1755,ST. LOUIS,NaN,36 inch bins,MINIATURE,ROUND TYPE,NaN,9/28/16,275.00,275.00,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,ABOUT STEADY.


# 预处理1-去除无用列

In [198]:
# data.columns

In [199]:
# 创建英文列名到中文列名的映射字典
column_mapping = {'City Name': '城市名称','Type': '类型','Package': '包装','Variety': '品种','Sub Variety': '子品种','Grade': '等级','Date': '日期','Low Price': '最低价格','High Price': '最高价格','Mostly Low': '主要最低价','Mostly High': '主要最高价', 'Origin': '产地','Origin District': '产地区域','Item Size': '物品尺寸','Color': '颜色','Environment': '环境','Unit of Sale': '销售单位','Quality': '质量','Condition': '状况','Appearance': '外观','Storage': '储存','Crop': '作物','Repack': '重新包装','Trans Mode': '运输模式','Unnamed: 24': '未命名: 24','Unnamed: 25': '未命名: 25'}
# 替换DataFrame的列名
data.rename(columns=column_mapping, inplace=True)
# 检查更改后的列名
print(data.columns.tolist())

['城市名称', '类型', '包装', '品种', '子品种', '等级', '日期', '最低价格', '最高价格', '主要最低价', '主要最高价', '产地', '产地区域', '物品尺寸', '颜色', '环境', '销售单位', '质量', '状况', '外观', '储存', '作物', '重新包装', '运输模式', '未命名: 24', '未命名: 25']


*重新包装:南瓜在销售前是否经过重新包装

In [200]:
from datetime import datetime
# 将日期列转换为日期格式
data['日期'] = pd.to_datetime(data['日期'], format='%m/%d/%y')
data["年份"] = data['日期'].dt.year
data["月份"] = data['日期'].dt.month
data["日"] = data['日期'].dt.day
data["星期"] = data['日期'].dt.weekday
data

,城市名称,类型,包装,品种,子品种,等级,日期,最低价格,最高价格,主要最低价,...,储存,作物,重新包装,运输模式,未命名: 24,未命名: 25,年份,月份,日,星期
0,BALTIMORE,NaN,24 inch bins,NaN,NaN,NaN,2017-04-29,270.00,280.00,270.0,...,NaN,NaN,E,NaN,NaN,NaN,2017,4,29,5
1,BALTIMORE,NaN,24 inch bins,NaN,NaN,NaN,2017-05-06,270.00,280.00,270.0,...,NaN,NaN,E,NaN,NaN,NaN,2017,5,6,5
2,BALTIMORE,NaN,24 inch bins,HOWDEN TYPE,NaN,NaN,2016-09-24,160.00,160.00,160.0,...,NaN,NaN,N,NaN,NaN,NaN,2016,9,24,5
3,BALTIMORE,NaN,24 inch bins,HOWDEN TYPE,NaN,NaN,2016-09-24,160.00,160.00,160.0,...,NaN,NaN,N,NaN,NaN,NaN,2016,9,24,5
4,BALTIMORE,NaN,24 inch bins,HOWDEN TYPE,NaN,NaN,2016-11-05,90.00,100.00,90.0,...,NaN,NaN,N,NaN,NaN,NaN,2016,11,5,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1752,ST. LOUIS,NaN,22 lb cartons,MINIATURE,FLAT TYPE,NaN,2016-09-30,14.75,14.75,NaN,...,NaN,NaN,N,NaN,NaN,LOWER.,2016,9,30,4
1753,ST. LOUIS,NaN,36 inch bins,MINIATURE,ROUND TYPE,NaN,2016-09-26,275.00,275.00,NaN,...,NaN,NaN,N,NaN,NaN,LOWER.,2016,9,26,0
1754,ST. LOUIS,NaN,36 inch bins,MINIATURE,ROUND TYPE,NaN,2016-09-27,275.00,275.00,NaN,...,NaN,NaN,N,NaN,NaN,STEADY.,2016,9,27,1
1755,ST. LOUIS,NaN,36 inch bins,MINIATURE,ROUND TYPE,NaN,2016-09-28,275.00,275.00,NaN,...,NaN,NaN,N,NaN,NaN,ABOUT STEADY.,2016,9,28,2


In [201]:
# 删除全部为空的列
data = data.dropna(axis=1, how='all')

In [202]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1757 entries, 0 to 1756
Data columns (total 21 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   城市名称     1757 non-null   object        
 1   类型       45 non-null     object        
 2   包装       1757 non-null   object        
 3   品种       1752 non-null   object        
 4   子品种      296 non-null    object        
 5   日期       1757 non-null   datetime64[ns]
 6   最低价格     1757 non-null   float64       
 7   最高价格     1757 non-null   float64       
 8   主要最低价    1654 non-null   float64       
 9   主要最高价    1654 non-null   float64       
 10  产地       1754 non-null   object        
 11  产地区域     131 non-null    object        
 12  物品尺寸     1478 non-null   object        
 13  颜色       1141 non-null   object        
 14  销售单位     162 non-null    object        
 15  重新包装     1757 non-null   object        
 16  未命名: 25  103 non-null    object        
 17  年份       1757 non-null   int32   

In [203]:
# 删除空值数量过多的列---该类数据的信息量太少，对后续分析无意义
data = data.dropna(thresh=len(data)*0.2, axis=1) #thresh参数:指定保留的行或列中至少应包含的非缺失值的数量(可以容忍的缺失值数量)

In [204]:
print(data['重新包装'].value_counts())
data[data['重新包装'] == 'E']

重新包装
N    1752
E       5
Name: count, dtype: int64


,城市名称,包装,品种,日期,最低价格,最高价格,主要最低价,主要最高价,产地,物品尺寸,颜色,重新包装,年份,月份,日,星期
0,BALTIMORE,24 inch bins,NaN,2017-04-29,270.0,280.0,270.0,280.0,MARYLAND,lge,NaN,E,2017,4,29,5
1,BALTIMORE,24 inch bins,NaN,2017-05-06,270.0,280.0,270.0,280.0,MARYLAND,lge,NaN,E,2017,5,6,5
29,BALTIMORE,24 inch bins,CINDERELLA,2017-06-03,380.0,380.0,380.0,380.0,NaN,xlge,NaN,E,2017,6,3,5
30,BALTIMORE,24 inch bins,CINDERELLA,2017-06-10,380.0,380.0,380.0,380.0,NaN,xlge,NaN,E,2017,6,10,5
31,BALTIMORE,24 inch bins,CINDERELLA,2017-06-17,380.0,380.0,380.0,380.0,NaN,xlge,NaN,E,2017,6,17,5


In [205]:
# 重新包装中N与E的数量比过大，E只有5条记录，且存在至少两个特征的缺失值，故不考虑重新包装特征
data.drop(columns=['重新包装'], inplace=True)

# 预处理2-空值填充-多种方法
注意，可以通过空值填充前后的两种方式对比，观察影响，选择最合适的方法

In [206]:
data

,城市名称,包装,品种,日期,最低价格,最高价格,主要最低价,主要最高价,产地,物品尺寸,颜色,年份,月份,日,星期
0,BALTIMORE,24 inch bins,NaN,2017-04-29,270.00,280.00,270.0,280.0,MARYLAND,lge,NaN,2017,4,29,5
1,BALTIMORE,24 inch bins,NaN,2017-05-06,270.00,280.00,270.0,280.0,MARYLAND,lge,NaN,2017,5,6,5
2,BALTIMORE,24 inch bins,HOWDEN TYPE,2016-09-24,160.00,160.00,160.0,160.0,DELAWARE,med,ORANGE,2016,9,24,5
3,BALTIMORE,24 inch bins,HOWDEN TYPE,2016-09-24,160.00,160.00,160.0,160.0,VIRGINIA,med,ORANGE,2016,9,24,5
4,BALTIMORE,24 inch bins,HOWDEN TYPE,2016-11-05,90.00,100.00,90.0,100.0,MARYLAND,lge,ORANGE,2016,11,5,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1752,ST. LOUIS,22 lb cartons,MINIATURE,2016-09-30,14.75,14.75,NaN,NaN,ILLINOIS,NaN,WHITE,2016,9,30,4
1753,ST. LOUIS,36 inch bins,MINIATURE,2016-09-26,275.00,275.00,NaN,NaN,MICHIGAN,sml,NaN,2016,9,26,0
1754,ST. LOUIS,36 inch bins,MINIATURE,2016-09-27,275.00,275.00,NaN,NaN,MICHIGAN,sml,NaN,2016,9,27,1
1755,ST. LOUIS,36 inch bins,MINIATURE,2016-09-28,275.00,275.00,NaN,NaN,MICHIGAN,sml,NaN,2016,9,28,2


In [207]:
data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1757 entries, 0 to 1756
Data columns (total 15 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   城市名称    1757 non-null   object        
 1   包装      1757 non-null   object        
 2   品种      1752 non-null   object        
 3   日期      1757 non-null   datetime64[ns]
 4   最低价格    1757 non-null   float64       
 5   最高价格    1757 non-null   float64       
 6   主要最低价   1654 non-null   float64       
 7   主要最高价   1654 non-null   float64       
 8   产地      1754 non-null   object        
 9   物品尺寸    1478 non-null   object        
 10  颜色      1141 non-null   object        
 11  年份      1757 non-null   int32         
 12  月份      1757 non-null   int32         
 13  日       1757 non-null   int32         
 14  星期      1757 non-null   int32         
dtypes: datetime64[ns](1), float64(4), int32(4), object(6)
memory usage: 178.6+ KB


### 热卡填充（Hot Deck Imputation）
>热卡填充是一种处理缺失数据的统计方法，核心思想是使用当前数据集中与缺失数据点相似的观测值来替代缺失值。这种方法的名称源自早期的计算机时代，当时数据存储在穿孔卡片上，"热卡"指的是当前正在处理的数据卡片。 
> 案例：填充某用户的 “教育程度” 缺失值时，找到年龄、职业相似且教育程度完整的用户，直接复制其值。

In [208]:
print('检查填充前空值情况：',data['品种'].isnull().sum()) 
# 使用空值记录中其他存在的特征相同的记录的品种 填充，如 城市名称、包装、产地、（颜色、尺寸）------ 使用其中出现频率最高的品种属性填充
data['品种'].fillna(data.groupby(['城市名称','包装','物品尺寸','产地','最低价格'])['品种'].transform(lambda x: x.mode().iloc[0] if not x.mode().empty else np.nan), inplace=True)
print('检查填充后是否还有空值：',data['品种'].isnull().sum()) # 检查填充后是否还有空值
data['品种'].fillna(data.groupby(['城市名称','包装','物品尺寸','产地'])['品种'].transform(lambda x: x.mode().iloc[0] if not x.mode().empty else np.nan), inplace=True)
print('放宽填要求后是否还有空值：',data['品种'].isnull().sum()) # 品牌成功填充

检查填充前空值情况： 5
检查填充后是否还有空值： 2
放宽填要求后是否还有空值： 0


In [210]:
print('检查填充前空值情况：',data['产地'].isnull().sum()) 
# 使用空值记录中其他存在的特征相同的记录的品种 填充，如 城市名称、包装、产地、（颜色、尺寸）------ 使用其中出现频率最高的品种属性填充
data['产地'].fillna(data.groupby(['城市名称','包装','物品尺寸','品种'])['产地'].transform(lambda x: x.mode().iloc[0] if not x.mode().empty else np.nan), inplace=True)
print('检查填充后是否还有空值：',data['产地'].isnull().sum())

检查填充前空值情况： 3
检查填充后是否还有空值： 0


In [156]:
data

,城市名称,包装,品种,日期,最低价格,最高价格,主要最低价,主要最高价,产地,物品尺寸,颜色,年份,月份,日,星期
0,BALTIMORE,24 inch bins,BIG MACK TYPE,2017-04-29,270.00,280.00,270.0,280.0,MARYLAND,lge,NaN,2017,4,29,5
1,BALTIMORE,24 inch bins,BIG MACK TYPE,2017-05-06,270.00,280.00,270.0,280.0,MARYLAND,lge,NaN,2017,5,6,5
2,BALTIMORE,24 inch bins,HOWDEN TYPE,2016-09-24,160.00,160.00,160.0,160.0,DELAWARE,med,ORANGE,2016,9,24,5
3,BALTIMORE,24 inch bins,HOWDEN TYPE,2016-09-24,160.00,160.00,160.0,160.0,VIRGINIA,med,ORANGE,2016,9,24,5
4,BALTIMORE,24 inch bins,HOWDEN TYPE,2016-11-05,90.00,100.00,90.0,100.0,MARYLAND,lge,ORANGE,2016,11,5,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1752,ST. LOUIS,22 lb cartons,MINIATURE,2016-09-30,14.75,14.75,NaN,NaN,ILLINOIS,NaN,WHITE,2016,9,30,4
1753,ST. LOUIS,36 inch bins,MINIATURE,2016-09-26,275.00,275.00,NaN,NaN,MICHIGAN,sml,NaN,2016,9,26,0
1754,ST. LOUIS,36 inch bins,MINIATURE,2016-09-27,275.00,275.00,NaN,NaN,MICHIGAN,sml,NaN,2016,9,27,1
1755,ST. LOUIS,36 inch bins,MINIATURE,2016-09-28,275.00,275.00,NaN,NaN,MICHIGAN,sml,NaN,2016,9,28,2


In [195]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1757 entries, 0 to 1756
Data columns (total 15 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   城市名称    1757 non-null   object        
 1   包装      1757 non-null   object        
 2   品种      1757 non-null   object        
 3   日期      1757 non-null   datetime64[ns]
 4   最低价格    1757 non-null   float64       
 5   最高价格    1757 non-null   float64       
 6   主要最低价   1654 non-null   float64       
 7   主要最高价   1654 non-null   float64       
 8   产地      1757 non-null   object        
 9   物品尺寸    1478 non-null   object        
 10  颜色      1141 non-null   object        
 11  年份      1757 non-null   int32         
 12  月份      1757 non-null   int32         
 13  日       1757 non-null   int32         
 14  星期      1757 non-null   int32         
dtypes: datetime64[ns](1), float64(4), int32(4), object(6)
memory usage: 178.6+ KB


In [94]:
data['物品尺寸'].value_counts()

物品尺寸
sml        363
med        337
lge        295
xlge       205
med-lge    133
jbo        128
exjbo       17
Name: count, dtype: int64

In [95]:
data.describe()


,日期,最低价格,最高价格,主要最低价,主要最高价,年份,月份,日,星期
count,1757,1757.000000,1757.000000,1654.000000,1654.000000,1757.000000,1757.000000,1757.000000,1757.000000
mean,2017-01-05 09:18:57.165623296,124.571338,132.970939,128.170550,132.491802,2016.249858,9.618099,17.602163,4.826409
min,2014-11-29 00:00:00,0.240000,0.240000,0.240000,0.240000,2014.000000,1.000000,1.000000,0.000000
25%,2016-10-08 00:00:00,24.000000,24.500000,24.625000,26.125000,2016.000000,9.000000,8.000000,5.000000
50%,2016-10-22 00:00:00,140.000000,150.000000,147.000000,150.000000,2016.000000,10.000000,22.000000,5.000000
75%,2017-01-28 00:00:00,180.000000,200.000000,185.000000,200.000000,2017.000000,10.000000,26.000000,5.000000
max,2017-09-30 00:00:00,480.000000,480.000000,480.000000,480.000000,2017.000000,12.000000,30.000000,5.000000
std,NaN,83.876591,89.524958,86.512161,88.442967,0.444732,1.104646,9.568983,0.775498


# 自定义任务设定
1. 哪些州的南瓜产量最多？不同地区的生产情况如何？哪些是地区特色/优势？
2. 哪里的南瓜价格最高？
3. 南瓜的大小与价格有何关系？
4. 哪种南瓜最贵？最便宜？
5. 不同月份的南瓜价格如何变化？南瓜市场上的不同类型南瓜数量如何变化？